In [9]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [10]:
import os
import sys
import time
import mahotas as mh
from skimage import measure
from tqdm.notebook import tqdm
from image.controller import ImageController

In [11]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
data_dir = '/raid/mpsych/RISTERLAB/fly_segmentation_experiments/data_dir/nina_d1_raw/'
save_dir = '/raid/mpsych/RISTERLAB/fly_segmentation_experiments/data_dir/04-14/nina_d1_mutant/binary_mask_5/'

In [13]:
LOGGING_IN_NOTEBOOK = True
log_path = save_dir + 'log.txt'
crop_path = save_dir + 'original_crop/'
binary_mask_path = save_dir + 'binary_mask/'
compare_plot_path = save_dir + 'compare_plots/'
compare_labeled_plot_path = save_dir + 'compare_labeled_plots/'

In [14]:
if not LOGGING_IN_NOTEBOOK:
    sys.stdout = open(log_path, 'w')

In [15]:
# NOTE: one single ommatidium has 7 rhabdomeres.
# Value NUM_RHABDOMERE and values related to threshold are not to be changed
NUM_RHABDOMERE = 7

THRESHOLD_BRACKET = 40
LEAST_POSSIBLE_THRESHOLD = 1
MOST_POSSIBLE_THRESHOLD = 255

# Change these values as per different datasets
# Values for Wild_Type_CS_1
#     CROP_SIZE = (512, 512)
#     MIN_REGION_SIZE = 750
#     MAX_REGION_SIZE = 750000
#     LEAST_PIXEL_COUNT = 20000
#     LEAST_OMMATIDIA_COUNT = 1
# Values for NINA_D1_MUTANT
#     CROP_SIZE = (512, 512)
#     MIN_REGION_SIZE = 150
#     MAX_REGION_SIZE = 750
#     LEAST_PIXEL_COUNT = 20000
#     LEAST_OMMATIDIA_COUNT = 12
CROP_SIZE = (512, 512)
MIN_REGION_SIZE = 150
MAX_REGION_SIZE = 750
LEAST_PIXEL_COUNT = 20000
LEAST_OMMATIDIA_COUNT = 6

In [16]:
is_mask_bad = True
good_mask_count, bad_mask_count = 0, 0
file_count = 0

for file in tqdm(sorted(os.listdir(data_dir))):
    start_time = time.time()
    
    if file.endswith(".tif"):
        
        num_rhabdomeres, num_ommatidia = 0, 0
        best_threshold, best_accuracy, threshold_value = 0, 0, 0
        
        file_count += 1
        print("Image Name: " + file)
        
        # Read file
        img_file = os.path.join(data_dir, file)
        data = ImageController.read(img_file)
        
        # Normalize and crop
        # crop size defaults to (512, 512)
        normalized_data = ImageController.normalize(data)
        crop_data = ImageController.center_crop(normalized_data, crop_size=CROP_SIZE)
        
        # Save original image cropped
        data_crop = ImageController.center_crop(data, crop_size=CROP_SIZE)
        ImageController.save(os.path.join(crop_path, file), data_crop)
        
        # Apply guassian filter
        # can pass sigma as argument, defaults to 3
        smoothed_data = ImageController.smooth(crop_data)
        
        # Playing around with threshold values.
        # Start with mean threshold
        # Checking accuracy for 40 threshold up and down the mean value
        # Has to be done due to contrast variation in data
        threshold_value = int(smoothed_data.mean())
        threshold_start = LEAST_POSSIBLE_THRESHOLD
        threshold_limit = MOST_POSSIBLE_THRESHOLD
        if (threshold_value - THRESHOLD_BRACKET > LEAST_POSSIBLE_THRESHOLD):
            threshold_start = int(threshold_value) - THRESHOLD_BRACKET
        if (threshold_value + THRESHOLD_BRACKET < MOST_POSSIBLE_THRESHOLD):
            threshold_limit = int(threshold_value) + THRESHOLD_BRACKET
        
        # Loop over all threshold values and get threshold with best accuracy
        for threshold in range(threshold_start, threshold_limit, 1):
            # Apply thresholding
            threshold_data = ImageController.threshold(smoothed_data, threshold_value=threshold)
            # region labelling
            labeled_data, nr_count = ImageController.label(threshold_data)
            # remove regions
            selected_labeled_data = ImageController.remove_regions(labeled_data, 
                                                                   min_region_size=MIN_REGION_SIZE, 
                                                                   max_region_size=MAX_REGION_SIZE)
            # Get binary mask of region removed labeled data
            binary_mask_data = ImageController.binary_mask(selected_labeled_data)
            # Apply close operation on binary mask
            segmented_image = ImageController.close_binary_mask(binary_mask_data)
            # count pixels in mask
            num_pixels = int(np.sum(segmented_image))
            # check accuracy of mask
            test_accuracy, test_num_rhabdomeres = ImageController.check_accuracy(binary_mask_data)
            # get best accuracy and satisfy least pixel count value
            if test_accuracy > best_accuracy and num_pixels > LEAST_PIXEL_COUNT:
                best_accuracy = test_accuracy
                best_threshold = threshold
                num_rhabdomeres = test_num_rhabdomeres
           
        # Run pipeline one last time for best threshold value
        threshold_data = ImageController.threshold(smoothed_data, threshold_value=best_threshold)
        labeled_data, nr_count = ImageController.label(threshold_data)
        selected_labeled_data = ImageController.remove_regions(labeled_data, 
                                                                   min_region_size=MIN_REGION_SIZE, 
                                                                   max_region_size=MAX_REGION_SIZE)
        binary_mask_data = ImageController.binary_mask(selected_labeled_data)
        closed_binary_mask_data = ImageController.close_binary_mask(binary_mask_data)
        segmented_image = ImageController.binary_image(closed_binary_mask_data)
        
        # count number of pixels in segmentation mask
        num_pixels = np.sum(segmented_image)
        
        # Save binary_mask, compare_labeled_plot and conpare_plot for best accuracy threshold
        compare_labeled_path = os.path.join(compare_labeled_plot_path, file)
        compare_labeled_plot_name = compare_labeled_path.split('.')[0] + '.png'
        
        compare_path = os.path.join(compare_plot_path, file)
        compare_plot_name = compare_path.split('.')[0] + '.png'
        
        binary_mask_name = os.path.join(binary_mask_path, file)
        
        # Save compare_labeled_plots for all images
        ImageController.display_and_save_compare_labeled(data_crop, 
                                                         binary_mask_data, 
                                                         segmented_image, 
                                                         compare_labeled_plot_name,
                                                         show_in_notebook=False)
        
        print("Threshold mean: ", smoothed_data.mean())
        print("Threshold best: ", best_threshold)
        print("Accuracy of mask: ", best_accuracy)
        
        num_ommatidia = round(num_rhabdomeres / NUM_RHABDOMERE)
        # if number of ommatidia in image and number of pixels in image
        # satisfy least count, then segmentation mask is good.
        if num_pixels > LEAST_PIXEL_COUNT and num_ommatidia >= LEAST_OMMATIDIA_COUNT:
            good_mask_count += 1
            print("Good segmentation mask")
            # Save binary mask and compare plot only for good mask image
            ImageController.save_compare(data_crop, segmented_image, compare_plot_name)
            ImageController.save(binary_mask_name, segmented_image)
        else:
            bad_mask_count += 1
            print("Bad segmentation mask")
            
        print("Possible number of rhabdomeres: ", num_rhabdomeres)
        print("Total number of pixels in the mask: ", num_pixels)
        
        end_time = time.time()
        print("Total Time Taken: ", (end_time - start_time), " seconds")
        
        print("Number of good masks so far: ", good_mask_count)
        print("Number of bad masks so far: ", bad_mask_count)
        print("Total files processed so far: ", file_count)
        
        print("---------------------------------------------------------")

print("Number of good masks: ", good_mask_count)
print("Number of bad masks: ", bad_mask_count)
print("Total processed images:", file_count)

  0%|          | 0/301 [00:00<?, ?it/s]

Image Name: 000000_000000_000903_000000.tif
Threshold mean:  29.143386126437907
Threshold best:  40
Accuracy of mask:  0.8688524590163934
Good segmentation mask
Possible number of rhabdomeres:  53
Total number of pixels in the mask:  20527
Total Time Taken:  2.993830680847168  seconds
Number of good masks so far:  1
Number of bad masks so far:  0
Total files processed so far:  1
---------------------------------------------------------
Image Name: 000000_000000_000905_000000.tif
Threshold mean:  29.24035932816866
Threshold best:  30
Accuracy of mask:  0.88
Good segmentation mask
Possible number of rhabdomeres:  44
Total number of pixels in the mask:  21031
Total Time Taken:  2.9802474975585938  seconds
Number of good masks so far:  2
Number of bad masks so far:  0
Total files processed so far:  2
---------------------------------------------------------
Image Name: 000000_000000_000906_000000.tif
Threshold mean:  29.497295056507273
Threshold best:  31
Accuracy of mask:  0.9166666666666

Threshold mean:  19.89148121875721
Threshold best:  19
Accuracy of mask:  0.8936170212765957
Good segmentation mask
Possible number of rhabdomeres:  42
Total number of pixels in the mask:  20395
Total Time Taken:  2.6456093788146973  seconds
Number of good masks so far:  20
Number of bad masks so far:  0
Total files processed so far:  20
---------------------------------------------------------
Image Name: 000000_000000_001003_000000.tif
Threshold mean:  16.875069874983566
Threshold best:  18
Accuracy of mask:  0.8928571428571429
Good segmentation mask
Possible number of rhabdomeres:  50
Total number of pixels in the mask:  21224
Total Time Taken:  2.539438247680664  seconds
Number of good masks so far:  21
Number of bad masks so far:  0
Total files processed so far:  21
---------------------------------------------------------
Image Name: 000000_000000_001005_000000.tif
Threshold mean:  18.203727273277316
Threshold best:  19
Accuracy of mask:  0.9444444444444444
Good segmentation mask

Threshold mean:  14.267521773502507
Threshold best:  15
Accuracy of mask:  0.921875
Good segmentation mask
Possible number of rhabdomeres:  59
Total number of pixels in the mask:  25078
Total Time Taken:  2.895782709121704  seconds
Number of good masks so far:  39
Number of bad masks so far:  0
Total files processed so far:  39
---------------------------------------------------------
Image Name: 000000_000000_001082_000000.tif
Threshold mean:  13.022487415062203
Threshold best:  13
Accuracy of mask:  0.9245283018867925
Good segmentation mask
Possible number of rhabdomeres:  49
Total number of pixels in the mask:  22599
Total Time Taken:  2.387951374053955  seconds
Number of good masks so far:  40
Number of bad masks so far:  0
Total files processed so far:  40
---------------------------------------------------------
Image Name: 000000_000000_001089_000000.tif
Threshold mean:  12.80494096514943
Threshold best:  12
Accuracy of mask:  0.94
Good segmentation mask
Possible number of rhabd

Threshold mean:  72.32770155637691
Threshold best:  85
Accuracy of mask:  0.92
Good segmentation mask
Possible number of rhabdomeres:  69
Total number of pixels in the mask:  20279
Total Time Taken:  3.532053232192993  seconds
Number of good masks so far:  58
Number of bad masks so far:  0
Total files processed so far:  58
---------------------------------------------------------
Image Name: 000000_000000_001157_000000.tif
Threshold mean:  71.21069531388335
Threshold best:  73
Accuracy of mask:  0.967741935483871
Good segmentation mask
Possible number of rhabdomeres:  60
Total number of pixels in the mask:  22976
Total Time Taken:  3.598186492919922  seconds
Number of good masks so far:  59
Number of bad masks so far:  0
Total files processed so far:  59
---------------------------------------------------------
Image Name: 000000_000000_001160_000000.tif
Threshold mean:  71.37760293614733
Threshold best:  72
Accuracy of mask:  0.9818181818181818
Good segmentation mask
Possible number o

...

Threshold mean:  32.65107714125525
Threshold best:  32
Accuracy of mask:  0.9354838709677419
Good segmentation mask
Possible number of rhabdomeres:  87
Total number of pixels in the mask:  26000
Total Time Taken:  3.1612422466278076  seconds
Number of good masks so far:  210
Number of bad masks so far:  0
Total files processed so far:  210
---------------------------------------------------------
Image Name: 000000_000000_001789_000000.tif
Threshold mean:  37.53415312522497
Threshold best:  37
Accuracy of mask:  0.9036144578313253
Good segmentation mask
Possible number of rhabdomeres:  75
Total number of pixels in the mask:  23534
Total Time Taken:  3.2921106815338135  seconds
Number of good masks so far:  211
Number of bad masks so far:  0
Total files processed so far:  211
---------------------------------------------------------
Image Name: 000000_000000_001791_000000.tif
Threshold mean:  38.75508038202923
Threshold best:  39
Accuracy of mask:  0.8571428571428571
Good segmentation m

Threshold mean:  61.73274895210407
Threshold best:  62
Accuracy of mask:  0.8860759493670886
Good segmentation mask
Possible number of rhabdomeres:  70
Total number of pixels in the mask:  21592
Total Time Taken:  3.4642655849456787  seconds
Number of good masks so far:  228
Number of bad masks so far:  1
Total files processed so far:  229
---------------------------------------------------------
Image Name: 000000_000000_001877_000000.tif
Threshold mean:  59.762888478708796
Threshold best:  62
Accuracy of mask:  0.9611650485436893
Good segmentation mask
Possible number of rhabdomeres:  99
Total number of pixels in the mask:  24961
Total Time Taken:  3.7154173851013184  seconds
Number of good masks so far:  229
Number of bad masks so far:  1
Total files processed so far:  230
---------------------------------------------------------
Image Name: 000000_000000_001879_000000.tif
Threshold mean:  52.22024133004549
Threshold best:  52
Accuracy of mask:  0.9186046511627907
Good segmentation 

Threshold mean:  46.002433737555705
Threshold best:  48
Accuracy of mask:  0.8620689655172413
Good segmentation mask
Possible number of rhabdomeres:  75
Total number of pixels in the mask:  21468
Total Time Taken:  3.4069647789001465  seconds
Number of good masks so far:  247
Number of bad masks so far:  1
Total files processed so far:  248
---------------------------------------------------------
Image Name: 000000_000000_001934_000000.tif
Threshold mean:  50.106615872610185
Threshold best:  53
Accuracy of mask:  0.9032258064516129
Good segmentation mask
Possible number of rhabdomeres:  84
Total number of pixels in the mask:  23453
Total Time Taken:  3.448730945587158  seconds
Number of good masks so far:  248
Number of bad masks so far:  1
Total files processed so far:  249
---------------------------------------------------------
Image Name: 000000_000000_001938_000000.tif
Threshold mean:  64.21310518600129
Threshold best:  67
Accuracy of mask:  0.8666666666666667
Good segmentation 

Threshold mean:  54.10690327179737
Threshold best:  0
Accuracy of mask:  0
Bad segmentation mask
Possible number of rhabdomeres:  0
Total number of pixels in the mask:  0
Total Time Taken:  2.868910551071167  seconds
Number of good masks so far:  257
Number of bad masks so far:  10
Total files processed so far:  267
---------------------------------------------------------
Image Name: 000000_000000_002009_000000.tif
Threshold mean:  64.88302632391235
Threshold best:  0
Accuracy of mask:  0
Bad segmentation mask
Possible number of rhabdomeres:  0
Total number of pixels in the mask:  0
Total Time Taken:  2.889376401901245  seconds
Number of good masks so far:  257
Number of bad masks so far:  11
Total files processed so far:  268
---------------------------------------------------------
Image Name: 000000_000000_002010_000000.tif
Threshold mean:  62.03217189041251
Threshold best:  0
Accuracy of mask:  0
Bad segmentation mask
Possible number of rhabdomeres:  0
Total number of pixels in th

Threshold mean:  43.06394457031083
Threshold best:  0
Accuracy of mask:  0
Bad segmentation mask
Possible number of rhabdomeres:  0
Total number of pixels in the mask:  0
Total Time Taken:  2.786250352859497  seconds
Number of good masks so far:  257
Number of bad masks so far:  30
Total files processed so far:  287
---------------------------------------------------------
Image Name: 000000_000000_002105_000000.tif
Threshold mean:  49.80774527155001
Threshold best:  0
Accuracy of mask:  0
Bad segmentation mask
Possible number of rhabdomeres:  0
Total number of pixels in the mask:  0
Total Time Taken:  2.7861480712890625  seconds
Number of good masks so far:  257
Number of bad masks so far:  31
Total files processed so far:  288
---------------------------------------------------------
Image Name: 000000_000000_002127_000000.tif
Threshold mean:  45.981087891610116
Threshold best:  0
Accuracy of mask:  0
Bad segmentation mask
Possible number of rhabdomeres:  0
Total number of pixels in 